# A full training

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate trl
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

zsh:1: no matches found: transformers[sentencepiece]


In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("sahil2801/CodeAlpaca-20k")
checkpoint = "Salesforce/codegen-350M-mono"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token=tokenizer.eos_token

# def tokenize_function(example):
#     return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

ConnectionError: Couldn't reach 'sahil2801/CodeAlpaca-20k' on the Hub (ConnectionError)

In [8]:
from trl import DataCollatorForCompletionOnlyLM

# # ref: https://huggingface.co/docs/trl/main/en/sft_trainer
# Same concept as code below but using DataCollatorForCompletionOnlyLM
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"Below is an instruction that describes a task. Write an output that appropriately completes the request.\n\n"
        text += f"### Instruction: {example['instruction'][i]}\n\n### Input: {example['input'][i]}\n\n### Output:\n{example['output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = "\n### Output:\n"
completionCollator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

/Users/parsa/lib/miniconda3/envs/torch-gpu/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


NameError: name 'tokenizer' is not defined

In [ ]:
def tokenize_function(example):
    prompt = formatting_prompts_func(example)
    return tokenizer(prompt, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) # , remove_columns=raw_datasets['train'].column_names

Map:   0%|          | 0/20022 [00:00<?, ? examples/s]

In [ ]:
# tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
# tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# tokenized_datasets.set_format("torch")
tokenized_datasets['train'].column_names

['input_ids', 'attention_mask']

In [ ]:
# ["attention_mask", "input_ids", "labels", "token_type_ids"]

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=completionCollator
)
# eval_dataloader = DataLoader(
#     tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
# )

In [2]:
next(iter(train_dataloader))

NameError: name 'train_dataloader' is not defined

In [3]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

NameError: name 'train_dataloader' is not defined

In [4]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(checkpoint)

NameError: name 'checkpoint' is not defined

In [10]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(1.7237, grad_fn=<NllLossBackward0>) torch.Size([8, 200, 51200])


In [11]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/Users/parsa/lib/miniconda3/envs/torch-gpu/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
from transformers import get_scheduler

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

2503


In [15]:
import torch

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)
device

device(type='mps')

In [16]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/2503 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
from peft import LoraConfig, get_peft_model
from peft import prepare_model_for_kbit_training

# TODO is it necessary?
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

#Lora config
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

peft_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    # QLoRA paper recommended to consider ALL linear layers.
    # Llama 2 uses the layers below
    # target_modules=[
    #     "query_key_value",
    #     "dense",
    #     "dense_h_to_4h",
    #     "dense_4h_to_h",
    # ]
)

model = get_peft_model(model, peft_config)

/Users/parsa/lib/miniconda3/envs/torch-gpu/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


NameError: name 'model' is not defined

In [21]:
from transformers import TrainingArguments

OUTPUT_DIR = 'lora-finetuned-' + checkpoint.split("/")[-1] + '-temp'
# OPTIM = "paged_adamw_32bit"


# Settings for T4 (A100)
MICRO_BATCH_SIZE = 2 # 4? for A100
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE # or just 4
EPOCHS = 1
LEARNING_RATE = 2e-4 #2e-5
CUTOFF_LEN = 256

WARMUP_RATIO = 0.03
MAX_STEPS = 100 # 300
MAX_GRAD_NORM = 0.3
LR_SCHEDULER_TYPE = "constant"
SAVE_STEPS = 50
LOGGING_STEPS = 5


training_args = TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    num_train_epochs=EPOCHS,
    # optim=OPTIM,
    warmup_ratio=WARMUP_RATIO, # warmup_steps=100,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    # fp16=True,
    # evaluation_strategy = "steps",
    save_steps= SAVE_STEPS, # save_total_limit=3,
    output_dir=OUTPUT_DIR,
    report_to='tensorboard',
    logging_steps=LOGGING_STEPS,
    logging_dir=OUTPUT_DIR+'/logs',
    # load_best_model_at_end=True, # requires evaluation strategy and save strategy to match
)


In [28]:
from trl import SFTTrainer

MAX_SEQ_LENGTH = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_datasets["train"],
    peft_config=peft_config,
    # dataset_text_field="text", # messy error
    formatting_func=formatting_prompts_func, # description above, data preparation
    data_collator=completionCollator,
    max_seq_length=MAX_SEQ_LENGTH,
    # tokenizer=tokenizer,
    args=training_args,
)

Map:   0%|          | 0/20022 [00:00<?, ? examples/s]

In [29]:
trainer.train()
trainer.save_model()

  0%|          | 0/156 [00:00<?, ?it/s]

KeyboardInterrupt: 

Evaluate

In [6]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

ModuleNotFoundError: No module named 'evaluate'

In [ ]:
print()

In [9]:
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

SyntaxError: '(' was never closed (2870412963.py, line 4)

In [ ]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(training_function)